![](http://blog.codec.wang/cv2_understand_padding.jpg)
## 卷积
什么是二维卷积呢？看下面一张图就一目了然
![](http://blog.codec.wang/cv2_understand_convolution.jpg)
卷积就是循环对图像跟一个核逐个元素相乘再求和得到另外一副图像的操作，比如结果图中第一个元素5是怎么算的呢？原图中3×3的区域与3×3的核逐个元素相乘再相加：

5=1×1+2×0+1×0+0×0+1×0+1×0+3×0+0×0+2×2

算完之后，整个框再往右移一步继续计算，横向计算完后，再往下移一步继续计算……网上有一副很经典的动态图，方便我们理解卷积：
![](http://blog.codec.wang/cv2_understand_cnn.gif)

## padding
不难发现，前面我们用3×3的核对一副6×6的图像进行卷积，得到的是4×4的图，图片缩小了！那怎么办呢？我们可以把原图扩充一圈，再卷积，这个操作叫填充padding。

事实上，原图为n×n，卷积核为f×f，最终结果图大小为(n-f+1) × (n-f+1)。
![](http://blog.codec.wang/cv2_understand_padding.jpg)
那么扩展的这一层应该填充什么值呢？OpenCV中有好几种填充方式，都使用cv2.copyMakeBorder()函数实现，一起来看看。

## 添加边框
cv2.copyMakeBorder()用来给图片添加边框，它有下面几个参数：
- src：要处理的原图
- top, bottom, left, right：上下左右要扩展的像素数
- borderType：边框类型，这个就是需要关注的填充方式，详情请参考：BorderTypes

其中默认方式和固定值方式最常用，我们详细说明一下：
### 固定值填充
顾名思义，cv2.BORDER_CONSTANT这种方式就是边框都填充成一个固定的值，比如下面的程序都填充0：

In [4]:
import cv2

img = cv2.imread('6_by_6.bmp', 0)
print(img)

cons = cv2.copyMakeBorder(img, 1, 1, 1, 1, cv2.BORDER_CONSTANT, value=0)
print(cons)

[[213 166 237 240 196 243]
 [166  81 213 181  34 197]
 [237 217 247 240 196 243]
 [245 200 241 241 199 240]
 [200  38 190 189  35 197]
 [241 185 237 240 189 241]]
[[  0   0   0   0   0   0   0   0]
 [  0 213 166 237 240 196 243   0]
 [  0 166  81 213 181  34 197   0]
 [  0 237 217 247 240 196 243   0]
 [  0 245 200 241 241 199 240   0]
 [  0 200  38 190 189  35 197   0]
 [  0 241 185 237 240 189 241   0]
 [  0   0   0   0   0   0   0   0]]


### 默认边框类型
默认边框cv2.BORDER_DEFAULT其实是取镜像对称的像素填充，比较拗口，一步步解释：

In [5]:
default = cv2.copyMakeBorder(img, 1, 1, 1, 1, cv2.BORDER_DEFAULT)
print(default)

[[ 81 166  81 213 181  34 197  34]
 [166 213 166 237 240 196 243 196]
 [ 81 166  81 213 181  34 197  34]
 [217 237 217 247 240 196 243 196]
 [200 245 200 241 241 199 240 199]
 [ 38 200  38 190 189  35 197  35]
 [185 241 185 237 240 189 241 189]
 [ 38 200  38 190 189  35 197  35]]


![](http://blog.codec.wang/cv2_up_down_padding_first.jpg)
同理再进行左右两边的填充，最后把四个顶点补充上就好了：
![](http://blog.codec.wang/cv2_right_left_padding_second2.jpg)

经验之谈：一般情况下默认方式更加合理，因为边界的像素值更加接近。具体应视场合而定。

OpenCV进行卷积
OpenCV中用cv2.filter2D()实现卷积操作，比如我们的核是下面这样（3×3区域像素的和除以10）：
```
        ⎡ 1 1 1 ⎤
M= 1/10 ⎢ 1 1 1 ⎥
        ⎣ 1 1 1 ⎦
```

In [8]:
import numpy as np
img = cv2.imread('lena.jpg')

kernel = np.ones((3, 3), np.float32)/10

dst = cv2.filter2D(img, -1, kernel)

TypeError: unhashable type: 'numpy.ndarray'

可以看到这个核对图像进行了模糊处理，这是卷积的众多功能之一。当然卷积还有很多知识没有学到，后面我们再继续深入。